In [ ]:
### Gerar tabelas das séries por ivs.

In [1]:
def dif(iv,micro,ano,f):
    import numpy as np
    from sklearn import preprocessing
    import pandas as pd
    import xuleta
    #Carregar dados
    datasmin = pd.read_hdf(banco, 'datas_min_micro')
    dados = pd.read_hdf(banco, 'ivs/micro/' + iv, converters={'data':str})
    base = pd.read_hdf(banco, 'base/micro/' + iv)
    
    # seta o índice para a base e datasmin
    base = base.set_index('period')
    datasmin = datasmin.set_index('year')
    
    #cria dicionario da microrregiao
    dicio = datasmin[micro].to_dict()
    
    #data inicial e final da serie de IV
    ini = dados[dados['data'] == dicio[ano]].index.tolist()
    inifin = [ini[0],ini[0]+20]
    
    #as series
    
    a = dados[micro][inifin[0]:inifin[1]].tolist()
    a = np.asarray(a)
    b = base[micro]

    #L = np.argmax(b) - np.argmax(a)
    #a = dados[micro][inifin[0]-L:inifin[1]-L]
    
    if iv == 'vci':
        L -= 1
        a = dados[micro][inifin[0]-L:inifin[1]-L]
    elif iv=='sdi':
        a = preprocessing.scale(a)
        b = preprocessing.scale(b)
    else:
        a=xuleta.dnorm(a)
        b=xuleta.dnorm(b)   
    
    suma = np.around(np.sum(a[0:15]),decimals=2)
    sumb = np.around(np.sum(b[0:15]),decimals=2)
    out = str(ano)+','+micro+','+iv+','+str(suma)+','+str(sumb)+','+str(suma-sumb)+'\n'
    print 'calculating ' + out
    f.write(out)
    

In [2]:
import numpy as np
banco = 'dados/banco_diss.h5'
ivs = ['tci']#,'vhi','sdi']#'ndvi','evi','ndi7','ndwi','lswi',
micros = ['COCP_CampoM', 'COCP_Goioere', 'NCP_Apucarana', 'NCP_Faxinal',
          'NCP_Florai','NCP_Ivaipora', 'NCP_Londrina', 'NCP_Maringa', 'NCP_Porecatu',
          'NP_Assai', 'NP_CornelioP','OP_Cascavel', 'OP_Foz', 'OP_Toledo']
anos = [i for i in np.arange(2002,2013,1)]

In [3]:
f = open('dados/tci.txt', 'a')
for iv in ivs:
    for micro in micros:
        for ano in anos:
            dif(iv,micro,ano,f)
f.close()

ImportError: No module named sklearn

Correlacionar dados
------------------

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, xuleta2
import seaborn as sn

In [5]:
NCP = ['NCP_Apucarana','NCP_Faxinal','NCP_Florai','NCP_Ivaipora', 
        'NCP_Londrina','NCP_Maringa','NCP_Porecatu']
OP = ['OP_Cascavel','OP_Foz','OP_Toledo']
NP = ['NP_Assai','NP_CornelioP']
COCP = ['COCP_CampoM','COCP_Goioere']

In [6]:
#dah pra limitar por mesorregiao usando where

#anos = [2006,2009,2010,2011,2012,2013]#anos molhados
#anos = [2003,2004,2005,2006,2007,2008,2009,2010,2011]#todos que em milho no ibge
anos = [2003,2004,2005,2007,2008,2012] #prec
#anos = [2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012]
#anos = [2005,2006,2007,2011] #spiJaneiro
#criterio = 'season = anos'
X = COCP + OP# + NCP + NP
criterio = 'region in X & season in anos'
dados = pd.read_hdf('dados/banco_diss.h5', 'full_statistics', where=criterio)

In [7]:
#listas
ldif = ['ndvi_dif','evi_dif','ndi7_dif','ndwi_dif','lswi_dif','vci_dif','tci_dif','vhi_dif','sdi_dif']
lprec = ['p_ondj','p_ndj','p_on','p_nd','p_dj','p_o','p_n','p_d','p_j']
lsafra = ['ndvi_safra','evi_safra','ndi7_safra','ndwi_safra',
          'lswi_safra','vci_safra','tci_safra','vhi_safra','sdi_safra']
lspi = ['spi3_sondjf','spi3_ndj','spi3_dj','spi3_n','spi3_d','spi3_j']
lprodfull = ['area_soy','prod_soy','yield_soy','area_maize','prod_maize', 
               'yield_maize','area_total','prod_total','yield_total']
lprod = ['yield_soy','yield_maize']#,'yield_maize_d','yield_maize_a']#,'yield_total']
livc= ['ndvi_dif','evi_dif','ndi7_dif','ndwi_dif','lswi_dif']

#datasets
prod = dados[ldif+lprod]
prodc = dados[livc+lprod]
spi = dados[ldif+lspi]
prec = dados[ldif+lprec]
safra = dados[lsafra+lprod]

In [8]:
safra

,ndvi_safra,evi_safra,ndi7_safra,ndwi_safra,lswi_safra,vci_safra,tci_safra,vhi_safra,sdi_safra,yield_soy,yield_maize
1,7.53,7.23,7.47,8.14,7.74,8.67,5.72,6.28,-6.62,3.07,5.67
2,6.95,6.54,6.88,7.17,6.96,6.74,8.14,6.38,5.88,2.89,6.48
3,7.84,7.03,7.95,7.48,8.29,8.28,5.26,6.18,4.15,2.43,6.57
5,7.04,6.30,6.75,7.16,6.97,7.91,7.07,4.14,6.54,3.36,7.87
6,8.05,7.36,7.88,8.03,8.14,5.59,6.06,6.70,3.19,3.02,8.09
10,7.41,5.95,6.95,7.00,6.89,7.76,9.54,8.19,7.23,2.55,9.09
12,8.03,7.97,8.02,7.41,8.13,7.66,6.70,7.67,-1.65,3.05,6.22
13,7.90,7.56,7.95,8.17,8.00,6.31,8.25,6.88,4.88,2.67,6.66
14,8.13,8.12,8.17,8.58,8.46,7.51,4.96,5.33,-0.21,2.44,6.27
16,8.05,7.50,8.00,8.21,8.16,7.35,7.01,5.61,6.02,2.64,5.87


In [78]:
out = '/home/denis/Copy/DissDOCS/dissertation/F_teste/'
f, ax = plt.subplots(figsize=(12, 12))
cmap = sn.diverging_palette(220,10, as_cmap=True)
sn.corrplot(prodc, annot=True, sig_stars=False,
            sig_corr=True, method='pearson',#
            cmap_range=(-.6,.60),
             diag_names=True, cmap=cmap, ax=ax)
f.tight_layout()
#plt.savefig(out+'Fcorr_ibgeallCOCPOP1.png', bbox_inches='tight', dpi=300, format='png')
plt.savefig(out+'Fcorr_ibgesecosCOCPOP1.pdf', bbox_inches='tight', format='pdf')
plt.show()
plt.close()